In [60]:
%load_ext autoreload 
%autoreload 2

from preprocessing import FileIO, Vectorizor
from opensearch_interface import OpenSearchClient
from index_templates import youtube_body

import os
import time
from rich import print
from dotenv import load_dotenv
load_env=load_dotenv('./.env', override=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Instantiate OpenSearch Client

In [61]:
model_path = 'sentence-transformers/all-MiniLM-L6-v2'
model_path_on_disk = os.environ['ST_MODEL_PATH']
osclient = OpenSearchClient(model_name_or_path=model_path)

### Display Existing Indices

In [62]:
osclient.show_indexes()

health status index                              uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   kw-impact-theory                   2MjMun4bQYOoeUpv5UsJxg   3   1      33164            0     29.4mb         29.4mb
yellow open   semantic-impact-theory-196         SY2nXyvmQ9i5LAS4hmn82g   3   1      37007            0    694.6mb        694.6mb
yellow open   kw-impact-theory-196               vsuHausxRb6EjysQriOX5w   3   1      37007            0     30.5mb         30.5mb
yellow open   security-auditlog-2023.10.21       Vj43Da3dTQm0mwBFNWHjCg   1   1          9            0    151.3kb        151.3kb
yellow open   security-auditlog-2023.10.22       YXYp6DkYT-aLgGxRZNGsUA   1   1       1704            0      1.6mb          1.6mb
yellow open   paul-graham3                       -74ZPvxoSMmtCPSzAI9o1A   1   1         18            0    768.2kb        768.2kb
yellow open   semantic-impact-theory-128         FJKOre3yT9aFxlF-_TvcTA   3   1      60380

Bad pipe message: %s [b'\xc4\x8d\x9aZI7\xdez\x11\xbb\x92\xbb\xa3\x990\xa7cg Nh\xd2]\x9f`WU]\xf1D\xe8Q\xaf\x07\xe1\xaf\xf9(\x9c|\x1a\x9d\xa9\xbf\x83\xa7\xdd:\x83jm\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01']
Bad pipe message: %s [b'']
Bad pipe message: %s [b"t\xcey\xe7Z\xad\xc6@1O'2\xe6^f\xd4vx\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d"]
Bad pipe message: %s [b"\x00\n\x8a\

### Load saved data from previous Ingest/Split/Vectorize process

In [29]:
data_path = './practice_data/impact_theory_minilm_196.parquet'

In [30]:
data = FileIO().load_parquet(data_path)

Shape of data: (37007, 16)
Memory Usage: 4.27+ MB


### Preview some data stats

In [37]:
data_keys = sorted(list(data[0].keys()))
emb_dim = len(data[0]['content_embedding'])
line_break = '-'*100
print(f'Data Keys: {data_keys}\n{line_break}\nEmbedding Dimension: {emb_dim}')

Data Keys: ['age_restricted', 'author', 'channel_id', 'content', 'content_embedding', 'description', 'doc_id', 'episode_num', 'keywords', 'length', 'playlist_id', 'publish_date', 'thumbnail_url', 'title', 'video_id', 'views']
----------------------------------------------------------------------------------------------------
Embedding Dimension: 384


### Data Indexing

In [19]:
index_name = 'impact-theory-minilm-196'

In [38]:
#Review Indexing Body
youtube_body

{'settings': {'number_of_shards': 3,
  'refresh_interval': '30s',
  'index': {'knn': True}},
 'mappings': {'properties': {'title': {'type': 'text', 'index': 'true'},
   'unique_id': {'type': 'keyword', 'index': 'false'},
   'group_id': {'type': 'short', 'index': 'false'},
   'video_id': {'type': 'keyword', 'index': 'false'},
   'playlist_id': {'type': 'keyword', 'index': 'false'},
   'episode_url': {'type': 'keyword', 'index': 'false'},
   'episode_num': {'type': 'short', 'index': 'false'},
   'description': {'type': 'text', 'index': 'true'},
   'length': {'type': 'long', 'index': 'false'},
   'publish_date': {'type': 'keyword', 'index': 'false'},
   'views': {'type': 'long', 'index': 'false'},
   'thumbnail_url': {'type': 'keyword', 'index': 'false'},
   'content': {'type': 'text', 'index': 'true'},
   'doc_id': {'type': 'keyword', 'index': 'false'},
   'content_embedding': {'type': 'knn_vector', 'dimension': 384}}}}

In [21]:
%%time
osclient.document_indexer(index_name=index_name, data=data, chunk_size=1500, body_template=youtube_body, semantic_index=True)
# time.sleep(30)

2023-10-21 21:47:45.022 | INFO     | opensearch_interface:document_indexer:212 - The ** impact-theory-minilm-196 ** index was created
2023-10-21 21:47:45.023 | INFO     | opensearch_interface:document_indexer:219 - The # of documents to be indexed = 37007
100%|█████████████████████████████████████████████████████████████████████████████████████████| 37007/37007 [00:12<00:00, 2870.34Docs Indexed/s]

CPU times: user 12.5 s, sys: 753 ms, total: 13.3 s
Wall time: 13.2 s


In [25]:
osclient.indices.refresh(index_name)

{'_shards': {'total': 6, 'successful': 3, 'failed': 0}}

## Test Search on New Index

In [27]:
query = 'How do I change my life for the better'

### Keyword Search

In [48]:
kw_response = osclient.keyword_search(query=query, index=index_name, size=5)
print(kw_response)

[
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'xdo1VIsB_9uU36NHek_k',
        '_score': 12.832875,
        '_source': {
            'length': 2548,
            'episode_num': 245,
            'title': 'Use These HACKS to UPGRADE Your Mindset',
            'thumbnail_url': 'https://i.ytimg.com/vi/vG9wxvLDTQE/hq720.jpg',
            'doc_id': 'vG9wxvLDTQE_2',
            'publish_date': '08-31-2021',
            'content': "How do I get the energy to push through man? You've got to find a way to find energy in 
that you can change So whatever position you're in right now You can change no matter what it is no matter how 
horrific no matter what you've been through no matter what you've done Like there is a way out from under it But 
part of it is allowing that sense of lightness to be there So I allowed it and so it began to lift off my shoulders
Which then encouraged me to take more action to read more to get better I started my obsession with gaining skills 
that reading isn't about checking a box. It's about actually Getting better at something that lets you do something
and I need a better way to explain this This is one of those things like if people really understood what I'm 
talking about right now This one fact would change their life forever.",
            'views': 344796,
            'video_id': 'vG9wxvLDTQE'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '_to1VIsB_9uU36NHZxSm',
        '_score': 12.465936,
        '_source': {
            'length': 7266,
            'episode_num': 351,
            'title': "Alzheimer's Disease: The EARLY WARNING SIGNS & How To Reverse It | Richard Johnson & Dale 
Bredesen",
            'thumbnail_url': 'https://i.ytimg.com/vi/Zn4MWA1BESY/hq720.jpg',
            'doc_id': 'Zn4MWA1BESY_41',
            'publish_date': '05-04-2023',
            'content': "It makes all the difference So for brain health You want a plant-rich? Ketogenic diet with 
appropriate periods of fasting so you want to get the ketones up you want to have the low-carb diet But you don't 
want to have just a pure meat diet. Why is that? So I'm ultra high I get most of my calories from meat. So if 
you're gonna So change my life right now. I want to hear about it. So what what is the difference? How how does it 
work mechanistically? Let's change your future for the better. Absolutely.",
            'views': 477365,
            'video_id': 'Zn4MWA1BESY'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'gdo1VIsB_9uU36NHiX7Z',
        '_score': 12.384281,
        '_source': {
            'length': 2329,
            'episode_num': 52,
            'title': 'How to Level Up in 2018',
            'thumbnail_url': 'https://i.ytimg.com/vi/4VMv1kJnDDs/hq720.jpg',
            'doc_id': '4VMv1kJnDDs_9',
            'publish_date': '01-02-2018',
            'content': "And it's really important to develop the skill, and I say that word purposefully, the skill
of knowing how to hear that inner wisdom and that intention kicking in and leaning into it quickly. So for me, my 
brain saying, that's it, right there, move as fast as a rocket, Mel. I wanted to change my life, and I think most 
people that are miserable or that are really like dying to be great and dying to have more, we want to change. We 
want to live a better life. We want to create more for our families. We want to be happier. The desire is there. 
Again, it's about how do you go from knowledge to action. So the first thing in the story that's important is 
realizing that the answer was in me, and my mind was telling me, pay attention. Could have also been the bourbon.",
            'views': 142117,
            'video_id': '4VMv1kJnDDs'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '9do1VIsB_9uU36NHjIvH',
        '_score': 12.2350025,
        '_source': {
            'length': 2995,
            'episode_num': 7,
            'title': 'DO 

### Vector Search

In [49]:
vec_response = osclient.vector_search(query, index_name, size=5)
print(vec_response)

[
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'H9o1VIsB_9uU36NHflyk',
        '_score': 0.55628854,
        '_source': {
            'length': 2962,
            'episode_num': 198,
            'title': "Psychotherapist's Hacks on How to Change Your Life | Lori Gottlieb on Impact Theory",
            'thumbnail_url': 'https://i.ytimg.com/vi/HKZa9rQxjts/hq720.jpg',
            'doc_id': 'HKZa9rQxjts_10',
            'publish_date': '11-17-2020',
            'content': "You're always doing these sort of little nudges, trying to get people to maybe recognize 
for themselves, have the epiphany and then sort of go off and change as a result of that epiphany. Why do people 
struggle so much with change? Change is really hard. I don't think people realize how hard it is because often when
they think of change, they think about making positive changes. And so you say to people, well, obviously making 
that change would improve your life. So why is it so hard to do? But then look at New Year's resolutions. We make 
those all the time. That's something that might improve our lives. And yet they don't tend to last. And the reason 
is because first of all, change is hard because change involves loss. So even though you might be moving to 
something better, what you do lose is you lose the familiar.",
            'views': 195847,
            'video_id': 'HKZa9rQxjts'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'v9o1VIsB_9uU36NHelPl',
        '_score': 0.5501879,
        '_source': {
            'length': 1859,
            'episode_num': 231,
            'title': 'CRUSH Your GOALS By Taking CONTROL of Your MIND and Your ACTIONS',
            'thumbnail_url': 'https://i.ytimg.com/vi/Ulk68JNcNlE/hq720.jpg',
            'doc_id': 'Ulk68JNcNlE_2',
            'publish_date': '06-12-2021',
            'content': "Why would you eliminate half of them? Fulfillment is all that matters. Don't chase money. 
Don't chase fame. Don't chase success. The only thing that matters to actually enjoying your life is to feel good 
about yourself when you're by yourself. That's it. If you are in a dark place and you hear change is real, change 
is possible, doesn't that lift your mood? Don't you want that to be true? Once you allow it, because you have to 
allow it, once you allow that to lift your mood and you're like, I feel better just by thinking I can change. That 
was huge for me. That was like moment one was when I realized I actually can change. So this thing that feels like 
a death threat, this like smothering cloud of just like despair, it can clear away.",
            'views': 134006,
            'video_id': 'Ulk68JNcNlE'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'Hdo1VIsB_9uU36NHbSug',
        '_score': 0.55005014,
        '_source': {
            'length': 5884,
            'episode_num': 317,
            'title': 'WHY YOUR LIFE IS SO BORING... (Fix This To Find Fulfillment) | Tom Bilyeu',
            'thumbnail_url': 'https://i.ytimg.com/vi/NoSkC1hn23Q/hq720.jpg',
            'doc_id': 'NoSkC1hn23Q_0',
            'publish_date': '01-28-2023',
            'content': "As you get older, you are less plastic. So it actually does become harder to learn new 
things. You begin to find the things that work for you. And so you think, oh, well, I know how to do this. And then
you encounter that moment where it stops working. And so now it's like, oh, wait a second. So Tom, we're going to 
bring it to the light later, but I'll start at the deep dark ends. What are the top five behaviors and mental traps
people do to live a really miserable life? What do people do? So ego is going to be number one. So the whole idea, 
if you want to live the good life, is to recognize that you are a deeply flawed individual that is holding yourself
back, that your life is a reflection entirely of your choices and not your circumstances.",
            'views': 97125,
      

### Hybrid Search

In [52]:
hyb_response = osclient.hybrid_search(query, index_name, index_name, kw_size=3, vec_size=3)
print(hyb_response)

[
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'xdo1VIsB_9uU36NHek_k',
        '_score': 12.832875,
        '_source': {
            'length': 2548,
            'episode_num': 245,
            'title': 'Use These HACKS to UPGRADE Your Mindset',
            'thumbnail_url': 'https://i.ytimg.com/vi/vG9wxvLDTQE/hq720.jpg',
            'doc_id': 'vG9wxvLDTQE_2',
            'publish_date': '08-31-2021',
            'content': "How do I get the energy to push through man? You've got to find a way to find energy in 
that you can change So whatever position you're in right now You can change no matter what it is no matter how 
horrific no matter what you've been through no matter what you've done Like there is a way out from under it But 
part of it is allowing that sense of lightness to be there So I allowed it and so it began to lift off my shoulders
Which then encouraged me to take more action to read more to get better I started my obsession with gaining skills 
that reading isn't about checking a box. It's about actually Getting better at something that lets you do something
and I need a better way to explain this This is one of those things like if people really understood what I'm 
talking about right now This one fact would change their life forever.",
            'views': 344796,
            'video_id': 'vG9wxvLDTQE'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'H9o1VIsB_9uU36NHflyk',
        '_score': 0.55628854,
        '_source': {
            'length': 2962,
            'episode_num': 198,
            'title': "Psychotherapist's Hacks on How to Change Your Life | Lori Gottlieb on Impact Theory",
            'thumbnail_url': 'https://i.ytimg.com/vi/HKZa9rQxjts/hq720.jpg',
            'doc_id': 'HKZa9rQxjts_10',
            'publish_date': '11-17-2020',
            'content': "You're always doing these sort of little nudges, trying to get people to maybe recognize 
for themselves, have the epiphany and then sort of go off and change as a result of that epiphany. Why do people 
struggle so much with change? Change is really hard. I don't think people realize how hard it is because often when
they think of change, they think about making positive changes. And so you say to people, well, obviously making 
that change would improve your life. So why is it so hard to do? But then look at New Year's resolutions. We make 
those all the time. That's something that might improve our lives. And yet they don't tend to last. And the reason 
is because first of all, change is hard because change involves loss. So even though you might be moving to 
something better, what you do lose is you lose the familiar.",
            'views': 195847,
            'video_id': 'HKZa9rQxjts'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': '_to1VIsB_9uU36NHZxSm',
        '_score': 12.465936,
        '_source': {
            'length': 7266,
            'episode_num': 351,
            'title': "Alzheimer's Disease: The EARLY WARNING SIGNS & How To Reverse It | Richard Johnson & Dale 
Bredesen",
            'thumbnail_url': 'https://i.ytimg.com/vi/Zn4MWA1BESY/hq720.jpg',
            'doc_id': 'Zn4MWA1BESY_41',
            'publish_date': '05-04-2023',
            'content': "It makes all the difference So for brain health You want a plant-rich? Ketogenic diet with 
appropriate periods of fasting so you want to get the ketones up you want to have the low-carb diet But you don't 
want to have just a pure meat diet. Why is that? So I'm ultra high I get most of my calories from meat. So if 
you're gonna So change my life right now. I want to hear about it. So what what is the difference? How how does it 
work mechanistically? Let's change your future for the better. Absolutely.",
            'views': 477365,
            'video_id': 'Zn4MWA1BESY'
        }
    },
    {
        '_index': 'impact-theory-minilm-196',
        '_id': 'v9o1VIsB_9uU36NHelPl',
        '_sc